In [1]:

if(!require('igraph')) install.packages('igraph')
library('igraph')

Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union




## Q16

In [135]:
graph <- read.graph("facebook_combined.txt", format="edgelist", directed=FALSE)
V(graph)$name <- V(graph)
subgraph <- induced_subgraph(graph, c(415,neighbors(fb_graph,415)))

Nr <- as.numeric(which(degree(subgraph)==24))

cat("the length of Nr is ", length(Nr), "\n")

the length of Nr is  11 


## Q17

In [140]:

common_algo <- function (graph, i, j) {
    return (length(intersect(neighbors(graph, i), neighbors(graph, j))))
}

jaccard_algo <- function (graph, i, j) {
    total_f <- length(union(neighbors(graph, i), neighbors(graph, j)))
    mutual_f <- length(intersect(neighbors(graph, i), neighbors(graph, j)))
    return (mutual_f/total_f)
}

adamic_algo <- function (graph, i, j) {
    result <- 0
    mutual_f <- intersect(neighbors(graph, i), neighbors(graph, j))
    for (node in mutual_f){
        result <- result + 1/log(length(neighbors(graph, node)))
    }
    return (result)
}



In [141]:
# recommend friend based on neighbor measure
friend_rec <- function (graph, i, size_of_deleted, algo = "jaccard") {
    # i, return_node: node index in V(graph), not node ID
    # t: recommemd t new friends
    
    friends_to_be_rec <- setdiff(V(graph), c(as.numeric(neighbors(graph, i)), i))
    value <- 0
    rec_value <- c()
    for (friend in friends_to_be_rec) {
        if (algo == "common") {
            value <- (common_algo(graph, i, friend))
        } else if (algo == "jaccard") {
            value <- (jaccard_algo(graph, i, friend))
        } else if (algo == "adamic") {
            value <- (adamic_algo(graph, i, friend))
        }
        rec_value <- c(rec_value, value)
    }
    
    return (friends_to_be_rec[sort(rec_value, decreasing=TRUE, index.return=TRUE)$ix[1:size_of_deleted]])
}

In [142]:

calculate_single_node_value <- function (graph, i, algo) {
    # delete current graph edges
    copy_graph <- graph
    deleted_f <- c()
    while (length(deleted_f) == 0) {
        for (node in as.numeric(neighbors(copy_graph, i))) {
            if (runif(1,0,1) <= 0.25) {
                copy_graph <- delete_edges(copy_graph, get.edge.ids(copy_graph,c(i,node)))
                deleted_f <- c(deleted_f, node)
            }
        }
    }
    
    # calculate value
    size_of_deleted = length(deleted_f)

    friends_to_be_rec <- setdiff(V(copy_graph), c(as.numeric(neighbors(copy_graph, i)), i))
    value <- 0
    rec_value <- c()
    for (friend in friends_to_be_rec) {
        if (algo == "common") {
            value <- (common_algo(copy_graph, i, friend))
        } else if (algo == "jaccard") {
            value <- (jaccard_algo(copy_graph, i, friend))
        } else if (algo == "adamic") {
            value <- (adamic_algo(copy_graph, i, friend))
        }
        rec_value <- c(rec_value, value)
    }  
    
    friends = friends_to_be_rec[sort(rec_value, decreasing=TRUE, index.return=TRUE)$ix[1:size_of_deleted]]
    
    return (length(intersect(friends, deleted_f))/length(deleted_f))
}

In [143]:
result <- data.frame()
for (algo in c("common", "jaccard", "adamic")){
    
    for (i in Nr){
        total_value <- 0
        for (n in 1:10) {
            total_value <- total_value + calculate_single_node_value(subgraph, i, algo)
        }
        value_aver <- total_value/10
        result[toString(V(subgraph)[i]$name), algo] <- value_aver
    }
}
result

,common,jaccard,adamic
,<dbl>,<dbl>,<dbl>
497,0.3566667,0.1365584,0.2538745
579,0.9909091,0.9547619,0.9857143
601,0.9365476,0.8386111,0.8606410
616,0.8529365,0.8316667,0.8004762
619,0.3976190,0.5025000,0.4620238
628,0.9875000,0.9246032,1.0000000
644,0.9421032,0.8622222,0.8371789
659,0.9857143,0.9888889,0.9466667
660,1.0000000,0.9708333,1.0000000


In [146]:
average_each_method <- colSums(result)/11

In [147]:
average_each_method

common   jaccard    adamic 
0.8462407 0.8057946 0.8179423